In [ ]:
import os
import re
import time
import uuid
from pyspark import SparkContext, SparkConf

# Adding AWS S3 Minio configs
sparkConf = (
    SparkConf()
    .set("spark.jars.ivy","/home/brijeshdhaker/.ivy2")
    .set("spark.ui.port", "4042")
    #.set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0,io.delta:delta-spark_2.12:3.3.2")
    #.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    #.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    #.set("spark.executor.heartbeatInterval", "300000")
    #.set("spark.network.timeout", "400000")
    #.set("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010")
    #.set("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO")
    #.set("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG")
    #.set("spark.hadoop.fs.s3a.path.style.access", "true")
    #.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.set("spark.eventLog.enabled", "true")
    #.set("spark.eventLog.dir", "file:///apps/var/logs/spark-events")
)
sc = SparkContext(master='local[*]', appName="wide-transformations", conf=sparkConf)
sc.setLogLevel('WARN')

In [ ]:
student_marks = [
    ("S-001","Hindi",456),("S-001","English",567),("S-001","Math",345),("S-001","Science",987),
    ("S-002","Hindi",878),("S-002","English",767),("S-002","Math",456),("S-002","Science",567),
    ("S-003","Hindi",630),("S-003","English",889),("S-003","Math",765),("S-003","Science",879),
    ("S-004","Hindi",879),("S-004","English",965),("S-004","Math",876),("S-004","Science",666),
    ("S-005","Hindi",567),("S-005","English",907),("S-005","Math",645),("S-005","Science",876),
    ("S-006","Hindi",776),("S-006","English",534),("S-006","Math",768),("S-006","Science",565),
    ("S-007","Hindi",556),("S-007","English",630),("S-007","Math",568),("S-007","Science",896),
    ("S-008","Hindi",765),("S-008","English",765),("S-008","Math",987),("S-008","Science",990),
    ("S-009","Hindi",777),("S-009","English",657),("S-009","Math",887),("S-009","Science",768),
    ("S-010","Hindi",648),("S-010","English",878),("S-010","Math",768),("S-010","Science",458),
    ("S-011","Hindi",878),("S-011","English",567),("S-011","Math",598),("S-011","Science",788),
    ("S-012","Hindi",779),("S-012","English",765),("S-012","Math",776),("S-012","Science",887),
    ("S-013","Hindi",887),("S-013","English",771),("S-013","Math",987),("S-013","Science",768),
    ("S-014","Hindi",677),("S-014","English",662),("S-014","Math",666),("S-014","Science",776),
    ("S-015","Hindi",550),("S-015","English",559),("S-015","Math",768),("S-015","Science",565),
    ("S-016","Hindi",625),("S-016","English",989),("S-016","Math",660),("S-016","Science",556)
]

In [ ]:
from pyspark.rdd import portable_hash
from random import random
import hashlib

keys = ["S-001","S-002","S-003","S-004","S-005","S-006","S-007","S-008","S-009","S-010","S-011","S-012","S-013","S-014","S-015","S-016"]

def hash_partitioning(key):
    return portable_hash(key) % len(keys)
    
    
def hash_partitioning(key):
    return hash(key) % len(keys)


def key_partitioning(key):
    return keys.index(key)


hashValue = lambda str: portable_hash(str)

In [ ]:
#
#
#
def print_partitions(rdd):
    numPartitions = rdd.getNumPartitions()
    print("Total partitions: {}".format(numPartitions))
    print("Partitioner: {}".format(rdd.partitioner))
    parts = rdd.glom().collect()
    i = 0
    j = 0
    for p in parts:
        print("================ Partition - {} ================".format(i))
        for r in p:
            print("Row {} : {} : {}".format(j, r, (hash(r[0])%numPartitions)))
            j = j+1
        i = i+1

In [ ]:
#
#
#
rawRDD = sc.parallelize(student_marks)
marksRDD = rawRDD.map(lambda x: (x[0], (x[1], x[2]))).partitionBy(4, key_partitioning)

# print("RDD-1 Partition Count : %i " % (rawRDD.getNumPartitions()))
# print("Values in RDD-1 : {0} ".format(rawRDD.glom().collect()))
print_partitions(marksRDD)

#### RDD.reduceByKey : Student's Total Marks

In [ ]:
from operator import add

print(marksRDD.map(lambda x: (x[0], x[1][1])).reduceByKey(add).collect())


#### RDD.groupByKey()

In [ ]:
print(marksRDD.groupByKey().glom().collect())

#### combineByKey

RDD.combineByKey(createCombiner, mergeValue, mergeCombiners, numPartitions=None, partitionFunc=<function portable_hash>)

In [ ]:

rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])

def to_list(a):
    return [a]

def append(a, b):
    a.append(b)
    return a

def extend(a, b):
    a.extend(b)
    return a

sorted(rdd.combineByKey(to_list, append, extend).collect())


#### aggregateByKey
RDD.aggregateByKey(zeroValue, seqFunc, combFunc, numPartitions=None, partitionFunc=<function portable_hash>)

In [ ]:
#
#
#

rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])

# Defining Sequential Operation and Combiner Operations
# Sequence operation for aggregation
def seq_agg(accumulator, element):
    return (accumulator[0]+ element, accumulator[1] + 1)


# Reduce/Combine operation for max aggregation#
def comb_agg(accumulator1, accumulator2):
    return (accumulator1[0] + accumulator2[0],accumulator1[1] + accumulator2[1])

seqFunc = (lambda x, y: (x[0] + y, x[1] + 1))
combFunc = (lambda x, y: (x[0] + y[0], x[1] + y[1]))

sorted(rdd.aggregateByKey((0, 0), seq_agg, comb_agg).collect())


#### Find Maximume for each key

In [ ]:
#
#
#

rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2),("a", 3), ("b", 14), ("a", 12)])

# Defining Sequential Operation and Combiner Operations
# Sequence operation for aggregation
def seq_agg(accumulator, element):
    min_val = min(accumulator["min"], element)
    max_val = max(accumulator["max"], element)
    return {"min":min_val, "max":max_val}


# Reduce/Combine operation for max aggregation#
def comb_agg(accumulator1, accumulator2):
    min_val = min(accumulator1["min"], accumulator2["min"])
    max_val = max(accumulator1["max"], accumulator2["max"])
    return {"min":min_val, "max":max_val}

print(rdd.aggregateByKey({"min":0, "max":0}, seq_agg, comb_agg).collect())

#### RDD.sortByKey()

In [ ]:
t_key_values = [
    ("ABC", 10.5),
    ("XYZ", 23.8),
    ("ABC", 11.2),
    ("XYZ", 20.3),
    ("ABC", 10.7),
    ("XYZ", 25.9),
    ("ABC", 15.1),
    ("PQR", 12.7),
    ("ABC", 12.5),
    ("PQR", 10.3),
    ("IJK", 12.5),
    ("LMN", 15.6)
]

rdd_1 = sc.parallelize(t_key_values)
print("RDD-1 Partition Count : %i " % (rdd_1.getNumPartitions()))
print("Values in RDD-1 : {0} ".format(rdd_1.collect()))

commons.print_separator()

#
rdd_2 = rdd_1.sortByKey(True)
print("Values in RDD-2 : {0} ".format(rdd_2.collect()))